# ANDES Power Flow Benchmark

Reference:

1. H. Cui, F. Li and K. Tomsovic, "Hybrid Symbolic-Numeric Framework for Power System Modeling and Analysis," in IEEE Transactions on Power Systems, vol. 36, no. 2, pp. 1373-1384, March 2021, doi: 10.1109/TPWRS.2020.3017019.
1. R. D. Zimmerman, C. E. Murillo-Sánchez and R. J. Thomas, "MATPOWER: Steady-State Operations, Planning, and Analysis Tools for Power Systems Research and Education," in IEEE Transactions on Power Systems, vol. 26, no. 1, pp. 12-19, Feb. 2011, doi: 10.1109/TPWRS.2010.2051168.
keywords: {Power system planning;Steady-state;Power system analysis computing;Power system simulation;Load flow;Open source software;Computer languages;Packaging;Costs;Mathematical model;Load flow analysis;optimal power flow;optimization methods;power engineering;power engineering education;power system economics;power system simulation;power systems;simulation software;software tools},
1. Power Grid Lib - Optimal Power Flow, https://github.com/power-grid-lib/pglib-opf

In [1]:
import platform
import os

from tqdm import tqdm
import numpy as np
import pandas as pd

import andes

import pypower.api as pyp

import datetime
import time

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# OS information
print(f"Python: {platform.python_version()}; OS: {platform.system()}")

print(f'andes:{andes.__version__}')

Last run time: 2024-03-26 17:35:23
Python: 3.9.18; OS: Darwin
andes:1.9.1.post24+g7a87ad5d


In [3]:
andes.config_logger(stream_level=50)

In [4]:
directory = '../../cases/matpower/'

# List all files and directories within the specified directory
entries = os.listdir(directory)

# Get a list of files in the directory along with their sizes
files_with_sizes = [(file, os.path.getsize(os.path.join(directory, file))) for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]

# Sort the list of files by size (the second element of the tuple)
sorted_files_with_sizes = sorted(files_with_sizes, key=lambda x: x[1])

# If you just want a list of filenames sorted by size
sorted_files = [file for file, size in sorted_files_with_sizes]

sorted_files.remove('.DS_Store')

In [5]:
n_cases = len(sorted_files)

res_cols = ['case', 'v_diff_max', 'a_diff_max', 't_ad', 't_pp', 'c_ad', 'c_pp']
res = pd.DataFrame(columns=res_cols)

for case_i in tqdm(range(n_cases)):
    pf_case = sorted_files[case_i]
    sa = andes.load(directory + pf_case,
                    setup=True, no_output=True, default_config=True)
    # --- PYPOWER ---
    ppc = andes.io.matpower.system2mpc(sa)
    ppopt = pyp.ppoption(VERBOSE=0, OUT_ALL=0, PF_ALG=1, OPF_ALG_DC=200)

    t_ad = time.time()
    sa.PFlow.run()
    s_ad = time.time() - t_ad

    t_pp = time.time()
    ppc_sol, _ = pyp.runpf(ppc, ppopt)
    s_pp = time.time() - t_pp

    if (sa.exit_code == 0) & (ppc_sol['success']):
        v_ad = sa.Bus.v.v
        a_ad = sa.Bus.a.v - sa.Bus.a.v[0]
        v_pp = ppc_sol['bus'][:, 7]
        a_pp = (ppc_sol['bus'][:, 8] - ppc_sol['bus'][0, 8]) * andes.shared.deg2rad

        v_diff = np.abs(v_ad - v_pp)
        a_diff = np.abs(a_ad - a_pp)
    else:
        v_diff = np.array([-1])
        a_diff = np.array([-1])

    row_res = {'case': pf_case.split('.')[0],
               'v_diff_max': np.max(v_diff),
               'a_diff_max': np.max(a_diff),
               't_ad': s_ad,
               't_pp': s_pp,
               'c_ad': int(1 - sa.exit_code),
               'c_pp': int(ppc_sol['success'])}
    res.loc[case_i] = row_res

100%|██████████| 42/42 [00:34<00:00,  1.22it/s]


In [6]:
res[['c_ad', 'c_pp']] = res[['c_ad', 'c_pp']].astype(bool)
res.round(6)

,case,v_diff_max,a_diff_max,t_ad,t_pp,c_ad,c_pp
0,case18,0.000000,0.000000,0.005228,0.008196,True,True
1,case22,-1.000000,-1.000000,0.009135,0.016628,True,False
2,case14,0.000000,0.000000,0.003891,0.005336,True,True
3,case30,0.000000,0.000000,0.004930,0.007562,True,True
4,case_ieee30,0.000000,0.000000,0.004656,0.006019,True,True
5,case69,-1.000000,-1.000000,0.003248,0.020533,False,False
6,case24_ieee_rts,0.000000,0.000000,0.005172,0.007628,True,True
7,case39,0.000000,0.000000,0.003234,0.003471,True,True
8,case85,-1.000000,-1.000000,0.003196,0.016355,False,False
9,case118zh,-1.000000,-1.000000,0.003179,0.017203,False,False
